In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy import io as spio




In [3]:
np.unique(np.array([1,2,3,2,2,3,4,5,3,4,3]), return_counts=True)

(array([1, 2, 3, 4, 5]), array([1, 3, 4, 2, 1]))

## Testing impurity

In [44]:
def impurity(left_label_hist, right_label_hist):
    """
    Calculates the impurity (badness) of a split. Uses entropy measure.
    :param left_label_hist: an array where key is 0 or 1, value is count
    :param right_label_hist:
    :return: impurity
    """
    #: modified log2 function to account for when proportion == 0
    def log2_modified(proportion):
        if proportion == 0:
            return 0
        else:
            return np.log2(proportion)


    total_count = left_label_hist[0] + right_label_hist[0] + left_label_hist[1] + right_label_hist[1]
    p_0_S = 1.0 * (left_label_hist[0] + right_label_hist[0]) / total_count #p_c is proportion of points in S that are in class C
    p_1_S = 1.0 * (left_label_hist[1] + right_label_hist[1]) / total_count
    H_S = -(p_0_S * log2_modified(p_0_S) + p_1_S * log2_modified(p_1_S)) # H(S), i.e. the entropy of S

    left_hist_count = (left_label_hist[0] + left_label_hist[1])
    if left_hist_count > 0:
        p_0_S_left = 1.0 * left_label_hist[0] / left_hist_count
        p_1_S_left = 1.0 * left_label_hist[1] / left_hist_count
        H_S_left = -(p_0_S_left * log2_modified(p_0_S_left) + p_1_S_left * log2_modified(p_1_S_left))  # H(S_l), i.e. the entropy of S_l
    else:
        H_S_left = 0
    
    right_hist_count = (right_label_hist[0] + right_label_hist[1])
    if right_hist_count > 0:    
        p_0_S_right = 1.0 * right_label_hist[0] / right_hist_count
        p_1_S_right = 1.0 * right_label_hist[1] / right_hist_count
        H_S_right = -(p_0_S_right * log2_modified(p_0_S_right) + p_1_S_right * log2_modified(p_1_S_right))  # H(S_l), i.e. the entropy of S_l
    else:
        H_S_right = 0
    
    H_after = (left_hist_count * H_S_left + right_hist_count * H_S_right) / total_count

    info_gain = H_S - H_after

    return -(info_gain)

In [45]:
left_label_hist = np.array([1,9])
right_label_hist = np.array([0,1])
print impurity(left_label_hist, right_label_hist)

-0.0131373563858


## Testing segmenter(.)

In [63]:
def segmenter(data, labels): #@@@
    numFeatures = data.shape[1]

    bestImpurity = float('inf')
    bestFeature = None # index of best feature
    bestThreshold = None # the elements with the bestThreshold value are in the right-hand set

    for i in np.arange(numFeatures):
        left_label_hist = np.array([0, 0])
        right_label_hist = np.array([labels.shape[0] - labels.sum(), labels.sum()])

        if isBinaryFeature[i] == False: # continuous feature #@@@
            sorted_vals = np.sort(data[:, i])
            indices = np.argsort(data[:, i])
            for j in np.arange(sorted_vals.shape[0]):
                if j != 0:
                    label_j1 = labels[indices[j-1]] # label of (j-1)th elem in the sorted_vals array
                    left_label_hist[label_j1] += 1
                    right_label_hist[label_j1] -= 1

                imp = impurity(left_label_hist, right_label_hist) #@@@
                if imp < bestImpurity:
                    bestImpurity = imp
                    bestFeature = i
                    bestThreshold = sorted_vals[j]
        else:  # binary feature
            for threshold in [0, 1]:
                if threshold == 1:
                    # add all elems with feature as value 0 (NOT LABEL) to left hist
                    for k in np.arange(data[:, i].shape[0]):
                        if data[k, i] == 0:
                            left_label_hist[labels[k]] += 1
                            right_label_hist[labels[k]] -= 1
                imp = impurity(left_label_hist, right_label_hist) #@@@
                if imp < bestImpurity:
                    bestImpurity = imp
                    bestFeature = i
                    bestThreshold = threshold

    if bestFeature == None or bestThreshold == None:
        print 'Error: in segmenter(.), bestFeature or bestThreshold were not found.'

    return bestFeature, bestThreshold

In [74]:
isBinaryFeature = [False, True, True]
data = np.array([[3, 1, 0],
                [3.5, 0, 1],
                [2.2, 1, 0],
                [3.1, 0, 1],
                [5, 0, 1],
                [1.2, 1, 0]])
labels = np.array([0, 1, 0, 1, 1, 0])

splitFeature, splitThreshold = segmenter(data, labels)

print splitThreshol

print data[3,0] == splitThreshold
print data[3,0]
print splitThreshold

3.1
True
3.1
3.1


In [78]:
np.zeros((4))

array([ 0.,  0.,  0.,  0.])